In [17]:
import glob

import iris
from iris.experimental.equalise_cubes import equalise_attributes

import xarray as xr

from dask.distributed import Client, progress, LocalCluster

In [9]:
client = LocalCluster() 
client

In [18]:
thetao_files = '/g/data/ua6/DRSv2/CMIP5/CSIRO-Mk3-6-0/piControl/mon/ocean/r1i1p1/thetao/latest/thetao_Omon_CSIRO-Mk3-6-0_piControl_r1i1p1_*.nc'
thetao_file_list = glob.glob(thetao_files)

## xarray

In [5]:
ds = xr.open_dataset(thetao_files)
thetao = ds['thetao']

In [6]:
theato

<xarray.DataArray 'pr' (time: 60, lat: 145, lon: 192)>
[1670400 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 9.375 11.25 13.12 15.0 ...
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 -85.0 -83.75 -82.5 ...
  * time     (time) datetime64[ns] 2001-01-16T12:00:00 2001-02-15 ...
Attributes:
    standard_name:     precipitation_flux
    long_name:         Precipitation
    units:             kg m-2 s-1
    comment:           at surface; includes both liquid and solid phases from...
    cell_methods:      time: mean
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

In [12]:
# the number of GB involved in the reduction
ds.thetao.nbytes/1e9

0.0066816

In [13]:
thetao.mean(dim=('lat','lon'))    #.load()

<xarray.DataArray 'pr' (time: 60)>
array([2.885244e-05, 2.942032e-05, 2.969076e-05, 2.937850e-05, 3.015959e-05,
       3.114218e-05, 3.141754e-05, 3.092697e-05, 3.025265e-05, 2.942208e-05,
       2.924221e-05, 2.898165e-05, 2.910258e-05, 2.900095e-05, 2.942142e-05,
       2.960631e-05, 3.091742e-05, 3.144607e-05, 3.122549e-05, 3.111620e-05,
       2.981876e-05, 2.996457e-05, 2.953239e-05, 2.911846e-05, 2.879192e-05,
       2.927267e-05, 2.956123e-05, 2.982080e-05, 3.015628e-05, 3.125701e-05,
       3.152196e-05, 3.092750e-05, 3.048339e-05, 2.977637e-05, 2.970339e-05,
       2.866807e-05, 2.915586e-05, 2.913593e-05, 2.950430e-05, 2.984615e-05,
       3.086244e-05, 3.166430e-05, 3.135809e-05, 3.135390e-05, 3.010619e-05,
       3.029122e-05, 2.919210e-05, 2.944928e-05, 2.907020e-05, 2.928065e-05,
       2.951659e-05, 2.971284e-05, 3.066567e-05, 3.119685e-05, 3.111589e-05,
       3.100750e-05, 3.098288e-05, 2.968451e-05, 2.997896e-05, 2.931432e-05],
      dtype=float32)
Coordinates:
  * ti

## iris

In [ ]:
cube = iris.load(thetao_files, 'sea_water_potential_temperature')
equalise_attributes(cube)
cube = cube.concatenate_cube()

In [ ]:
cube = cube.collapsed(['longitude', 'latitude'], iris.analysis.MEAN)